# Federated Learning with Pytorch
## A Pytorch implementation of Federated Learning Algorithm FedSGD on MNIST dataset

Federated Learning is a machine learning setting where many clients (e.g. mobile devices or whole organizations) collaboratively train a model under the orchestration of a central server (e.g. service provider), while keeping the training data decentralized. Federated Learning represents a possible solution to the problem of data privacy and data security in machine learning. 

I implemented the Federated Learning algorithm FedSGD on the MNIST dataset using Pytorch. The algorithm is implemented in the `fedSgd` function. The `fedSgdPar` function is a parallel version where a thread is spawn for each client. The `Client` class represents a client in the federated learning system. The `Server` class represents the server in the federated learning system. The `models` class contains the neural network model implementation that I tested.

In [ ]:
from src.fedsgd import fedSgdPar, fedSgdSeq

fedSgdPar()

Round 0: 100%|██████████| 100/100 [00:40<00:00,  6.69it/s, loss=2.28]

##### TODO 
- [ ] Add implementation code description: 
	- [ ] `Client` class
	- [ ] `Server` class
	- [ ] `models` class
	- [ ] `fedSgdPar` and `fedSgd` algorithm (show the algorithm code)
- [ ] Run experiment (from the paper)
- [ ] Show results
- [x] Code refactoring: move the class in separate files and the main code in a separate file